In [1]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision/research'

In [2]:
%cd ../

/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision


In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    include_top: bool
    trainable: bool
    input_shape: list
    freeze_all: bool
    freeze_till: int
    learning_rate: float
    classes: int

In [12]:
from cnnClassifier.constants import *
import os
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params

        prepare_base_model = PrepareBaseModelConfig(
            root_dir=config.root_dir, 
            base_model_path=config.base_model_path, 
            updated_base_model_path=config.updated_base_model_path, 
            include_top=params.INCLUDE_TOP, 
            trainable=params.TRAINABLE, 
            input_shape=params.INPUT_SHAPE, 
            freeze_all=params.FREEZE_ALL, 
            freeze_till=params.FREEZE_TILL, 
            learning_rate=params.LEARNING_RATE, 
            classes=params.CLASSES
        )

        return prepare_base_model

In [13]:
import tensorflow as tf
from tensorflow.keras import layers

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def get_base_model(self):
        self.model = tf.keras.applications.EfficientNetB0(include_top=self.config.include_top)

        self.save_model(self.config.base_model_path, self.model)
    @staticmethod
    def _prepare_full_model(base_model, input_shape, freeze_all, freeze_till, classes, learning_rate):
        if freeze_all:
            base_model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in base_model.layers[:-freeze_till]:
                base_model.trainable = False
        
        inputs = layers.Input(shape=input_shape, name="input_layer")
        
        x = base_model(inputs, training=False)
        x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
        x = layers.Dense(classes)(x)

        outputs = layers.Activation("softmax", dtype=tf.float32, name="siftmax_float32")(x)
        full_model = tf.keras.Model(inputs, outputs)

        full_model.compile(loss="sparse_categorical_crossentropy", 
                           optimizer=tf.keras.optimizers.Adam(), 
                           metrics=["accuracy"])
        
        full_model.summary()
        return full_model
        
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            base_model=self.model, 
            input_shape=self.config.input_shape, 
            freeze_all=self.config.freeze_all, 
            freeze_till=self.config.freeze_till, 
            classes=self.config.classes, 
            learning_rate=self.config.learning_rate
        )

        self.save_model(self.config.updated_base_model_path, self.full_model)





In [14]:
from cnnClassifier import logger

STAGE_NAME = "Prepare Base Model"

try:
    logger.info(f">>>>>>> stage {STAGE_NAME} started <<<<<<<<<<")
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    logger.info(f">>>>>>> stage {STAGE_NAME} completed <<<<<<<<\n\nx=============x")

except Exception as e:
    raise e


[2023-10-10 09:30:04,356: INFO: 596458800: >>>>>>> stage Prepare Base Model started <<<<<<<<<<]
[2023-10-10 09:30:04,363: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-10 09:30:04,369: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-10 09:30:04,371: INFO: common: created directory at: artifacts]
[2023-10-10 09:30:06,928: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


/home/ebi/miniconda3/envs/tf_full/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1280)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
 siftmax_float32 (Activatio  (None, 10)                0         
 n)                                                              
                                                             